In [30]:
!pip install hazm
!pip install summa
!pip install persian_wordcloud

In [31]:
import sys
import pandas as pd
import numpy as np
import csv
import re   
import hazm
from summa import keywords
from persian_wordcloud.wordcloud import PersianWordCloud

In [32]:
lemm = hazm.Lemmatizer()
normalizer = hazm.Normalizer()

In [33]:
def preprocess(df, stopwords):
    df['ptext'] = df['text'].apply(lambda x : re.sub(r'[\s]{2,}',' ',x))
    df['ptext'] = df.ptext.apply(lambda x : x.replace("\n"," "))    
    df["ptext"] = df.ptext.apply(lambda x : ' '.join([normalizer.normalize(word) for word in hazm.word_tokenize(x)]))    
    df['ptext'] = df.ptext.apply(lambda x :' '.join([lemm.lemmatize(word) for word in hazm.word_tokenize(x)]))
    df['ptext'] = df['ptext'].apply(lambda x : re.sub('[^\u0600-\u06FF\s]',' ',x))
    df['ptext'] = df.ptext.apply(lambda x : ' '.join([word for word in hazm.word_tokenize(x) if word not in stopwords]))
    return df

In [34]:
dic_key = []
df = pd.read_excel('lyrics.xlsx')
stopwords = pd.read_csv('stopwords.txt',
                        sep='\n',
                        quotechar=None,
                        quoting=csv.QUOTE_NONE,
                        header=None)[0].tolist()

In [35]:
df = preprocess(df, stopwords)
array_text = df.ptext.values
kewords_list=[]

In [36]:
for j in range(len(array_text)):
    keys = (keywords.keywords(array_text[j], words=5)).split("\n")
    kewords_list.extend(keys)
#         print("Keywords of article", str(j+1), "\n", keys)

In [37]:
# dic_key=pd.Series(kewords_list).value_counts().to_dict()
# print("len of kewords_list is {} and count of unique word is {} ".format(len(kewords_list),len(dic_key)))

In [38]:
def removeWeirdChars(text):
    weridPatterns = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u'\U00010000-\U0010ffff'
                               u"\u200d"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\u3030"
                               u"\ufe0f"
                               u"\u2069"
                               u"\u2066"
                               u"\u200c"
                               u"\u2068"
                               u"\u2067"
                               "]+", flags=re.UNICODE)
    return weridPatterns.sub(r'', text)

In [39]:
stopwords = ['n','یه', 'تو','دیگه','تورو','دارم','میشه','رو','چی','ای','کن','منو','من','بگو','اون','خوئم','واسه','تویی','وقتی','نیست','کنم','داری','آخه','منی','خودم','خودت','نمیشه','نیست','باشه','باشی','باشه','شه','میشم','ماند','','','','','','','','']

def remove_general_stopwords(text):
    filtered_tokens = [token for token in text if token not in stopwords]
    filtered_text = ' '.join(filtered_tokens)
    return filtered_text

kewords_list = remove_general_stopwords(kewords_list)

all_text =''
for word in kewords_list:
    all_text += word

# Generate a word cloud image
wordcloud = PersianWordCloud(
        only_persian=True,
        max_words=100,
        margin=0,
        width=800,
        height=800,
        min_font_size=1,
        max_font_size=500,
        background_color="white").generate(all_text)
image = wordcloud.to_image()
image.save("res.png")